## Conclusion from SQL and panda pre analysis
### Observation
* The code of a control area is unique. A control area will only have one unit and multiple SCPs but units may be duplicate among stations. Linename is messy. => Just use Control Area
* Time is not consistent too. => Use morning/afternoon/night...
* Get needed columns to save almost 100 mb
* Can keep desc-recovered but will need to clean duplicate data with both regular and recovered
* SQL query may be faster

### Completed
* Query data from database
* Clean duplicate data with both regular and recovered
* Create datetime frame
* Substract entries and exits to get the numbers

### To do
* Clean data
* Substract entries and exits by time
* Time analysis

In [1]:
#*****need to run*****
from sqlalchemy import create_engine
#from sqlalchemy import inspect
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

In [2]:
#*****need to run*****
engine = create_engine("sqlite:///mta.db")

In [3]:
#*****need to run*****
#df = pd.read_sql('SELECT ca, scp, station, date, time, desc, entries, exits FROM mta_data;', engine)
df = pd.read_sql('''SELECT ca, scp, station, date, time, desc, entries, exits FROM mta_data
                    WHERE date NOT IN ("02/27/2021", "02/28/2021", "06/01/2021", 
                                       "06/02/2021", "06/03/2021", "06/04/2021")
                    ORDER BY ca, date, time, scp, desc;''', engine)

In [4]:
#*****need to run*****
df["o_index"] = df.index
df["o_index"]
#drop duplicate data with wrong recover audit first
mask_dup = df[(df["o_index"] == 300706) |
          (df["o_index"] == 1089956) |
          (df["o_index"] == 1382146) |
          (df["o_index"] == 1382154) |
          ((df["CA"] == "N100") &
           (df["SCP"] == "00-00-00") &
           (df["ENTRIES"] == 818272) &
           (df["o_index"] != 842617))].index
df.drop(mask_dup, inplace= True)
#then, drop duplicate data with wrong regular data
df.drop_duplicates(subset=["CA", "SCP", "STATION", "DATE", "TIME"], keep= 'last', inplace=True)
df.drop(['o_index'], axis = 1, inplace = True)
df.reset_index(drop = True, inplace = True)
#datatime series
df["DATE_TIME"] = pd.to_datetime(df.DATE + " " + df.TIME, format="%m/%d/%Y %H:%M:%S")
df["DAY"] = df["DATE_TIME"].dt.weekday

In [6]:
#*****need to run*****
df.sort_values(["CA", "SCP","DATE_TIME"], inplace=True)
df["PRE_ENTRIES"] = df.ENTRIES.shift(1)
df["PRE_EXITS"] = df.EXITS.shift(1)
df["PRE_TIME"] = df.DATE_TIME.shift(1)
df["PERIOD_ENTRIES"] = df.ENTRIES.diff(1)
df["PERIOD_EXITS"] = df.EXITS.diff(1)
df.dropna(inplace=True)
df.drop(df[(df["PERIOD_ENTRIES"] < 0) | (df["PERIOD_EXITS"] < 0)].index, inplace=True)
df.drop(df[(df["DATE_TIME"] < df["PRE_TIME"])].index, inplace=True)
df.drop(df[((df["CA"] == "PTH03") & (df["SCP"] == "00-01-08") & (df["PRE_EXITS"] == 0.0))].index, inplace=True)
df.drop(df[(df["DATE_TIME"] - df["PRE_TIME"] > timedelta(days = 1))].index, inplace=True)
df.drop(df[(df["PERIOD_ENTRIES"] > 10000) | (df["PERIOD_EXITS"] > 10000)].index, inplace=True)
df["PERIOD_VISITS"] = df["PERIOD_ENTRIES"] + df["PERIOD_EXITS"]
df.reset_index(drop = True, inplace = True)

In [32]:
#too complicated and genarate NAN
'''
df["PERIOD_ENTRIES_MEAN"] = ((df.groupby(["CA","SCP", "TIME"])
                                .PERIOD_ENTRIES
                                .transform(lambda x: (x.mean() + 3 * x.std()))))
'''

In [7]:
df.head(10)

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,DAY,PRE_ENTRIES,PRE_EXITS,PRE_TIME,PERIOD_ENTRIES,PERIOD_EXITS,PERIOD_VISITS
0,A002,02-00-00,59 ST,03/01/2021,07:00:00,REGULAR,7537862,2570742,2021-03-01 07:00:00,0,7537860.0,2570718.0,2021-03-01 03:00:00,2.0,24.0,26.0
1,A002,02-00-00,59 ST,03/01/2021,11:00:00,REGULAR,7537914,2570858,2021-03-01 11:00:00,0,7537862.0,2570742.0,2021-03-01 07:00:00,52.0,116.0,168.0
2,A002,02-00-00,59 ST,03/01/2021,15:00:00,REGULAR,7538016,2570907,2021-03-01 15:00:00,0,7537914.0,2570858.0,2021-03-01 11:00:00,102.0,49.0,151.0
3,A002,02-00-00,59 ST,03/01/2021,19:00:00,REGULAR,7538294,2570952,2021-03-01 19:00:00,0,7538016.0,2570907.0,2021-03-01 15:00:00,278.0,45.0,323.0
4,A002,02-00-00,59 ST,03/01/2021,23:00:00,REGULAR,7538400,2570968,2021-03-01 23:00:00,0,7538294.0,2570952.0,2021-03-01 19:00:00,106.0,16.0,122.0
5,A002,02-00-00,59 ST,03/02/2021,03:00:00,REGULAR,7538409,2570969,2021-03-02 03:00:00,1,7538400.0,2570968.0,2021-03-01 23:00:00,9.0,1.0,10.0
6,A002,02-00-00,59 ST,03/02/2021,07:00:00,REGULAR,7538419,2570995,2021-03-02 07:00:00,1,7538409.0,2570969.0,2021-03-02 03:00:00,10.0,26.0,36.0
7,A002,02-00-00,59 ST,03/02/2021,11:00:00,REGULAR,7538469,2571124,2021-03-02 11:00:00,1,7538419.0,2570995.0,2021-03-02 07:00:00,50.0,129.0,179.0
8,A002,02-00-00,59 ST,03/02/2021,15:00:00,REGULAR,7538572,2571162,2021-03-02 15:00:00,1,7538469.0,2571124.0,2021-03-02 11:00:00,103.0,38.0,141.0
9,A002,02-00-00,59 ST,03/02/2021,19:00:00,REGULAR,7538821,2571195,2021-03-02 19:00:00,1,7538572.0,2571162.0,2021-03-02 15:00:00,249.0,33.0,282.0


In [32]:
df[df["PERIOD_ENTRIES"] > 2000].sort_values("PERIOD_ENTRIES").head(10)

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,DAY,PRE_ENTRIES,PRE_EXITS,PRE_TIME,PERIOD_ENTRIES,PERIOD_EXITS
1676438,PTH20,03-01-08,NEWARK HM HE,03/29/2021,13:53:22,REGULAR,3977,1045806,2021-03-29 13:53:22,0,647.0,30613.0,2021-03-29 12:31:05,3330.0,1015193.0
1681669,PTH20,03-01-08,NEWARK HM HE,05/19/2021,14:09:23,REGULAR,5109,1148469,2021-05-19 14:09:23,2,784.0,31332.0,2021-05-19 10:26:15,4325.0,1117137.0
209713,B020,00-00-00,AVENUE H,04/05/2021,12:59:12,REGULAR,56092,57421,2021-04-05 12:59:12,0,41866.0,18983.0,2021-04-05 12:00:00,14226.0,38438.0
1560538,PTH03,00-01-08,JOURNAL SQUARE,03/17/2021,13:15:45,REGULAR,21538,15082,2021-03-17 13:15:45,2,232.0,156.0,2021-03-17 09:44:02,21306.0,14926.0
1699052,PTH22,00-00-00,PATH NEW WTC,04/22/2021,11:30:36,REGULAR,21708,93657,2021-04-22 11:30:36,3,2.0,0.0,2021-04-22 10:07:34,21706.0,93657.0
2228394,R258,00-00-00,125 ST,05/12/2021,17:00:00,REGULAR,44859,21100,2021-05-12 17:00:00,2,12719.0,13431.0,2021-05-12 13:00:00,32140.0,7669.0
2314844,R315,01-00-01,PROSPECT AV,04/30/2021,13:00:00,REGULAR,55606,16813861,2021-04-30 13:00:00,4,8662.0,71310.0,2021-04-30 09:00:00,46944.0,16742551.0
1902121,R161B,00-05-02,72 ST,05/08/2021,13:00:00,REGULAR,393216,0,2021-05-08 13:00:00,5,327680.0,0.0,2021-05-08 09:00:00,65536.0,0.0
801871,N091,02-05-03,CHAMBERS ST,04/10/2021,00:00:00,RECOVR AUD,393416,196634,2021-04-10 00:00:00,5,302266.0,76419.0,2021-04-09 12:00:00,91150.0,120215.0
1608359,PTH07,00-00-00,CITY / BUS,05/30/2021,06:07:20,REGULAR,164591,132283,2021-05-30 06:07:20,6,47214.0,105866.0,2021-05-30 04:34:41,117377.0,26417.0


In [28]:
df[df["PERIOD_EXITS"] > 2000].sort_values("PERIOD_EXITS").head(20)

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,DAY,PRE_ENTRIES,PRE_EXITS,PRE_TIME,PERIOD_ENTRIES,PERIOD_EXITS
2522876,R528,00-00-04,JUNCTION BLVD,03/13/2021,19:00:00,REGULAR,3815638,1497072,2021-03-13 19:00:00,5,3815267.0,1495065.0,2021-03-13 15:00:00,371.0,2007.0
2198008,R249,01-00-09,86 ST,04/16/2021,16:00:00,REGULAR,2321988,22606153,2021-04-16 16:00:00,4,2321860.0,22604013.0,2021-04-16 08:00:00,128.0,2140.0
842572,N100,00-00-00,HIGH ST,03/06/2021,15:00:00,RECOVR AUD,818272,1731687,2021-03-06 15:00:00,5,816762.0,1728271.0,2021-03-06 11:00:00,1510.0,3416.0
2228394,R258,00-00-00,125 ST,05/12/2021,17:00:00,REGULAR,44859,21100,2021-05-12 17:00:00,2,12719.0,13431.0,2021-05-12 13:00:00,32140.0,7669.0
1560538,PTH03,00-01-08,JOURNAL SQUARE,03/17/2021,13:15:45,REGULAR,21538,15082,2021-03-17 13:15:45,2,232.0,156.0,2021-03-17 09:44:02,21306.0,14926.0
1608359,PTH07,00-00-00,CITY / BUS,05/30/2021,06:07:20,REGULAR,164591,132283,2021-05-30 06:07:20,6,47214.0,105866.0,2021-05-30 04:34:41,117377.0,26417.0
1608382,PTH07,00-00-00,CITY / BUS,05/30/2021,10:19:20,REGULAR,164597,132293,2021-05-30 10:19:20,6,47214.0,105868.0,2021-05-30 08:46:41,117383.0,26425.0
1608405,PTH07,00-00-00,CITY / BUS,05/30/2021,14:31:20,REGULAR,164605,132317,2021-05-30 14:31:20,6,47222.0,105877.0,2021-05-30 12:58:41,117383.0,26440.0
1608474,PTH07,00-00-00,CITY / BUS,05/31/2021,03:07:20,REGULAR,164610,132374,2021-05-31 03:07:20,0,47229.0,105923.0,2021-05-31 01:34:41,117381.0,26451.0
1608451,PTH07,00-00-00,CITY / BUS,05/30/2021,22:55:20,REGULAR,164610,132369,2021-05-30 22:55:20,6,47228.0,105907.0,2021-05-30 21:22:41,117382.0,26462.0


In [30]:
mask = ((df["CA"] == "R258") &
        (df["SCP"] == "00-00-00") &
        (df["DATE"] == "05/12/2021"))
df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,DAY,PRE_ENTRIES,PRE_EXITS,PRE_TIME,PERIOD_ENTRIES,PERIOD_EXITS
2228342,R258,00-00-00,125 ST,05/12/2021,01:00:00,REGULAR,12472,12496,2021-05-12 01:00:00,2,12471.0,12322.0,2021-05-11 21:00:00,1.0,174.0
2228355,R258,00-00-00,125 ST,05/12/2021,05:00:00,REGULAR,12472,12521,2021-05-12 05:00:00,2,12472.0,12496.0,2021-05-12 01:00:00,0.0,25.0
2228368,R258,00-00-00,125 ST,05/12/2021,09:00:00,REGULAR,12620,12973,2021-05-12 09:00:00,2,12472.0,12521.0,2021-05-12 05:00:00,148.0,452.0
2228381,R258,00-00-00,125 ST,05/12/2021,13:00:00,REGULAR,12719,13431,2021-05-12 13:00:00,2,12620.0,12973.0,2021-05-12 09:00:00,99.0,458.0
2228394,R258,00-00-00,125 ST,05/12/2021,17:00:00,REGULAR,44859,21100,2021-05-12 17:00:00,2,12719.0,13431.0,2021-05-12 13:00:00,32140.0,7669.0
2228407,R258,00-00-00,125 ST,05/12/2021,21:00:00,REGULAR,44861,21541,2021-05-12 21:00:00,2,44859.0,21100.0,2021-05-12 17:00:00,2.0,441.0


In [16]:
mask = ((df["CA"] == "PTH03") &
        (df["SCP"] == "00-01-08") &
        (df["DATE"] == "03/17/2021"))
df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,DAY,PRE_ENTRIES,PRE_EXITS,PRE_TIME,PERIOD_ENTRIES,PERIOD_EXITS
1560460,PTH03,00-01-08,JOURNAL SQUARE,03/17/2021,00:39:45,REGULAR,21409,15049,2021-03-17 00:39:45,2,21397.0,15036.0,2021-03-16 20:27:45,12.0,13.0
1560485,PTH03,00-01-08,JOURNAL SQUARE,03/17/2021,04:51:45,REGULAR,21415,15054,2021-03-17 04:51:45,2,21409.0,15049.0,2021-03-17 00:39:45,6.0,5.0
1560511,PTH03,00-01-08,JOURNAL SQUARE,03/17/2021,09:03:45,REGULAR,21494,15064,2021-03-17 09:03:45,2,21415.0,15054.0,2021-03-17 04:51:45,79.0,10.0
1560538,PTH03,00-01-08,JOURNAL SQUARE,03/17/2021,13:15:45,REGULAR,21538,15082,2021-03-17 13:15:45,2,232.0,156.0,2021-03-17 09:44:02,21306.0,14926.0
1560564,PTH03,00-01-08,JOURNAL SQUARE,03/17/2021,17:27:45,REGULAR,21568,15099,2021-03-17 17:27:45,2,21538.0,15082.0,2021-03-17 13:15:45,30.0,17.0
1560590,PTH03,00-01-08,JOURNAL SQUARE,03/17/2021,21:39:45,REGULAR,21594,15116,2021-03-17 21:39:45,2,21568.0,15099.0,2021-03-17 17:27:45,26.0,17.0


In [7]:
mask = ((df["CA"] == "PTH07") &
        (df["DATE"] == "05/30/2021")) 
df[mask].sort_values("SCP").head(20)

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,DAY
1608382,PTH07,00-00-00,CITY / BUS,05/30/2021,10:19:20,REGULAR,164597,132293,2021-05-30 10:19:20,6
1608400,PTH07,00-00-00,CITY / BUS,05/30/2021,12:58:41,REGULAR,47222,105877,2021-05-30 12:58:41,6
1608354,PTH07,00-00-00,CITY / BUS,05/30/2021,04:34:41,REGULAR,47214,105866,2021-05-30 04:34:41,6
1608405,PTH07,00-00-00,CITY / BUS,05/30/2021,14:31:20,REGULAR,164605,132317,2021-05-30 14:31:20,6
1608377,PTH07,00-00-00,CITY / BUS,05/30/2021,08:46:41,REGULAR,47214,105868,2021-05-30 08:46:41,6
1608446,PTH07,00-00-00,CITY / BUS,05/30/2021,21:22:41,REGULAR,47228,105907,2021-05-30 21:22:41,6
1608451,PTH07,00-00-00,CITY / BUS,05/30/2021,22:55:20,REGULAR,164610,132369,2021-05-30 22:55:20,6
1608428,PTH07,00-00-00,CITY / BUS,05/30/2021,18:43:20,REGULAR,164609,132354,2021-05-30 18:43:20,6
1608359,PTH07,00-00-00,CITY / BUS,05/30/2021,06:07:20,REGULAR,164591,132283,2021-05-30 06:07:20,6
1608423,PTH07,00-00-00,CITY / BUS,05/30/2021,17:10:41,REGULAR,47224,105892,2021-05-30 17:10:41,6


In [52]:
df.loc[746405, "DATE_TIME"] - df.loc[746405, "PRE_TIME"] > timedelta(days = 1)

True

In [23]:
mask = ((df["CA"] == "PTH03") &
        (df["SCP"] == "00-01-08") &
        (df["PRE_EXITS"] == 0.0))
df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,DAY,PRE_ENTRIES,PRE_EXITS,PRE_TIME,PERIOD_ENTRIES,PERIOD_EXITS
1558178,PTH03,00-01-08,JOURNAL SQUARE,03/01/2021,09:56:47,REGULAR,19111,14055,2021-03-01 09:56:47,0,1.0,0.0,2021-03-01 08:42:00,19110.0,14055.0
1563329,PTH03,00-01-08,JOURNAL SQUARE,04/05/2021,11:48:11,REGULAR,24642,16337,2021-04-05 11:48:11,0,1.0,0.0,2021-04-05 07:38:03,24641.0,16337.0
1563926,PTH03,00-01-08,JOURNAL SQUARE,04/09/2021,12:24:11,REGULAR,25390,16620,2021-04-09 12:24:11,4,1.0,0.0,2021-04-09 09:50:16,25389.0,16620.0
1565012,PTH03,00-01-08,JOURNAL SQUARE,04/16/2021,21:16:11,REGULAR,26237,17112,2021-04-16 21:16:11,4,1.0,0.0,2021-04-16 17:56:40,26236.0,17112.0
1565802,PTH03,00-01-08,JOURNAL SQUARE,04/22/2021,07:42:11,REGULAR,26854,17451,2021-04-22 07:42:11,3,1.0,0.0,2021-04-22 07:18:04,26853.0,17451.0
1565981,PTH03,00-01-08,JOURNAL SQUARE,04/23/2021,13:06:11,REGULAR,27063,17527,2021-04-23 13:06:11,4,1.0,0.0,2021-04-23 12:57:49,27062.0,17527.0
1570543,PTH03,00-01-08,JOURNAL SQUARE,05/24/2021,09:01:48,REGULAR,1,0,2021-05-24 09:01:48,0,1.0,0.0,2021-05-24 08:24:42,0.0,0.0
1570548,PTH03,00-01-08,JOURNAL SQUARE,05/24/2021,09:42:11,REGULAR,31986,19635,2021-05-24 09:42:11,0,1.0,0.0,2021-05-24 09:01:48,31985.0,19635.0


In [6]:
dup = (df
       .groupby(["CA", "SCP", "STATION", "DATE", "TIME"])
       .ENTRIES.count()
       .reset_index()
       .sort_values("ENTRIES"))
dup[dup.ENTRIES > 1].shape

(0, 6)

In [8]:
df.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2712197 entries, 0 to 2712196
Data columns (total 16 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   CA              2712197 non-null  object        
 1   SCP             2712197 non-null  object        
 2   STATION         2712197 non-null  object        
 3   DATE            2712197 non-null  object        
 4   TIME            2712197 non-null  object        
 5   DESC            2712197 non-null  object        
 6   ENTRIES         2712197 non-null  int64         
 7   EXITS           2712197 non-null  int64         
 8   DATE_TIME       2712197 non-null  datetime64[ns]
 9   DAY             2712197 non-null  int64         
 10  PRE_ENTRIES     2712197 non-null  float64       
 11  PRE_EXITS       2712197 non-null  float64       
 12  PRE_TIME        2712197 non-null  datetime64[ns]
 13  PERIOD_ENTRIES  2712197 non-null  float64       
 14  PERIOD_EXITS    27

In [9]:
for col in df:
    print(col, df[col].nunique())

CA 750
SCP 222
STATION 379
DATE 92
TIME 56712
DESC 2
ENTRIES 1775443
EXITS 1686012
DATE_TIME 173829
DAY 7
PRE_ENTRIES 1774964
PRE_EXITS 1685812
PRE_TIME 173826
PERIOD_ENTRIES 927
PERIOD_EXITS 1404
PERIOD_VISITS 1593


In [10]:
for col in df:
    print(df[col].value_counts())

PTH22    23484
R610     13771
PTH03    13475
PTH02    12143
PTH16    12030
         ...  
H037      1102
J022      1102
N215      1099
A085       551
OB01       307
Name: CA, Length: 750, dtype: int64
00-00-01    249238
00-00-00    247705
00-00-02    222957
00-00-03    113157
00-03-00     98253
             ...  
03-01-08       521
01-00-0A       521
00-04-09       520
02-02-04       518
03-01-07       506
Name: SCP, Length: 222, dtype: int64
34 ST-PENN STA     55662
FULTON ST          52896
GRD CNTRL-42 ST    47955
23 ST              39672
86 ST              35394
                   ...  
225 ST              1104
CLEVELAND ST        1102
SUTTER AV           1102
182-183 STS         1099
ORCHARD BEACH        307
Name: STATION, Length: 379, dtype: int64
04/12/2021    30004
03/19/2021    29839
03/16/2021    29812
04/07/2021    29763
04/05/2021    29724
              ...  
05/05/2021    29467
04/20/2021    29467
05/14/2021    29423
03/14/2021    26871
03/01/2021    24597
Name: DATE, Lengt

In [10]:
#*********must run*********
#turnstiles_daily = (df.groupby(["CA", "SCP", "STATION", "DATE"], as_index=False).ENTRIES.last())
#(df.groupby(["CA", "SCP", "STATION", "DATE"], as_index=False).ENTRIES.max())
daily_max = (df
             .groupby(["CA", "SCP", "STATION", "DATE"], as_index=False)
             .agg({"ENTRIES":"max", "EXITS":"max"})
             .rename(columns={"ENTRIES": "ENTRIES_MAX", "EXITS": "EXITS_MAX"}))
daily_max

,CA,SCP,STATION,DATE,ENTRIES_MAX,EXITS_MAX
0,A002,02-00-00,59 ST,03/01/2021,7538400,2570968
1,A002,02-00-00,59 ST,03/02/2021,7538926,2571211
2,A002,02-00-00,59 ST,03/03/2021,7539503,2571472
3,A002,02-00-00,59 ST,03/04/2021,7540040,2571750
4,A002,02-00-00,59 ST,03/05/2021,7540626,2572022
...,...,...,...,...,...,...
459351,TRAM2,00-05-01,RIT-ROOSEVELT,05/27/2021,5554,577
459352,TRAM2,00-05-01,RIT-ROOSEVELT,05/28/2021,5554,577
459353,TRAM2,00-05-01,RIT-ROOSEVELT,05/29/2021,5554,577
459354,TRAM2,00-05-01,RIT-ROOSEVELT,05/30/2021,5554,577


In [11]:
#*********must run*********
#turnstiles_daily = (df.groupby(["CA", "SCP", "STATION", "DATE"], as_index=False).ENTRIES.first())
#turnstiles_daily = (df.groupby(["CA", "SCP", "STATION", "DATE"], as_index=False).ENTRIES.min())
daily_min = (df
             .groupby(["CA", "SCP", "STATION", "DATE"], as_index=False)
             .agg({"ENTRIES":"min", "EXITS":"min"})
             .rename(columns={"ENTRIES": "ENTRIES_MIN", "EXITS": "EXITS_MIN"}))
daily_min

,CA,SCP,STATION,DATE,ENTRIES_MIN,EXITS_MIN
0,A002,02-00-00,59 ST,03/01/2021,7537860,2570718
1,A002,02-00-00,59 ST,03/02/2021,7538409,2570969
2,A002,02-00-00,59 ST,03/03/2021,7538938,2571214
3,A002,02-00-00,59 ST,03/04/2021,7539514,2571474
4,A002,02-00-00,59 ST,03/05/2021,7540050,2571750
...,...,...,...,...,...,...
459351,TRAM2,00-05-01,RIT-ROOSEVELT,05/27/2021,5554,577
459352,TRAM2,00-05-01,RIT-ROOSEVELT,05/28/2021,5554,577
459353,TRAM2,00-05-01,RIT-ROOSEVELT,05/29/2021,5554,577
459354,TRAM2,00-05-01,RIT-ROOSEVELT,05/30/2021,5554,577


In [12]:
#*********must run*********
#SCP daily level
daily = pd.merge(daily_max, daily_min, on = ["CA", "SCP", "STATION", "DATE"])
daily["ENTRIES_DAILY"] = daily["ENTRIES_MAX"] - daily["ENTRIES_MIN"]
daily["EXITS_DAILY"] = daily["EXITS_MAX"] - daily["EXITS_MIN"]
daily["SUM_DAILY"] = daily["ENTRIES_DAILY"] + daily["EXITS_DAILY"]
daily["DATE"] = pd.to_datetime(daily.DATE, format="%m/%d/%Y")
daily["DAY"] = daily["DATE"].dt.weekday
daily

,CA,SCP,STATION,DATE,ENTRIES_MAX,EXITS_MAX,ENTRIES_MIN,EXITS_MIN,ENTRIES_DAILY,EXITS_DAILY,SUM_DAILY,DAY
0,A002,02-00-00,59 ST,2021-03-01,7538400,2570968,7537860,2570718,540,250,790,0
1,A002,02-00-00,59 ST,2021-03-02,7538926,2571211,7538409,2570969,517,242,759,1
2,A002,02-00-00,59 ST,2021-03-03,7539503,2571472,7538938,2571214,565,258,823,2
3,A002,02-00-00,59 ST,2021-03-04,7540040,2571750,7539514,2571474,526,276,802,3
4,A002,02-00-00,59 ST,2021-03-05,7540626,2572022,7540050,2571750,576,272,848,4
...,...,...,...,...,...,...,...,...,...,...,...,...
459351,TRAM2,00-05-01,RIT-ROOSEVELT,2021-05-27,5554,577,5554,577,0,0,0,3
459352,TRAM2,00-05-01,RIT-ROOSEVELT,2021-05-28,5554,577,5554,577,0,0,0,4
459353,TRAM2,00-05-01,RIT-ROOSEVELT,2021-05-29,5554,577,5554,577,0,0,0,5
459354,TRAM2,00-05-01,RIT-ROOSEVELT,2021-05-30,5554,577,5554,577,0,0,0,6


In [13]:
daily[daily["ENTRIES_DAILY"] < 0]

,CA,SCP,STATION,DATE,ENTRIES_MAX,EXITS_MAX,ENTRIES_MIN,EXITS_MIN,ENTRIES_DAILY,EXITS_DAILY,SUM_DAILY,DAY


In [14]:
daily[daily["EXITS_DAILY"] < 0]

,CA,SCP,STATION,DATE,ENTRIES_MAX,EXITS_MAX,ENTRIES_MIN,EXITS_MIN,ENTRIES_DAILY,EXITS_DAILY,SUM_DAILY,DAY


In [15]:
#*********must run*********
mask_outlier = daily[(daily["ENTRIES_DAILY"] < 0) |
                     (daily["EXITS_DAILY"] < 0) |
                     (daily["ENTRIES_DAILY"] > 30000) |
                     (daily["EXITS_DAILY"] > 30000)].index
daily.drop(mask_outlier, inplace= True)
daily.sort_values("SUM_DAILY", ascending=False)

,CA,SCP,STATION,DATE,ENTRIES_MAX,EXITS_MAX,ENTRIES_MIN,EXITS_MIN,ENTRIES_DAILY,EXITS_DAILY,SUM_DAILY,DAY
283334,PTH22,00-01-04,PATH NEW WTC,2021-03-05,26234,24186,45,222,26189,23964,50153,4
261642,PTH03,00-01-08,JOURNAL SQUARE,2021-04-23,27171,17555,1,0,27170,17555,44725,4
261641,PTH03,00-01-08,JOURNAL SQUARE,2021-04-22,26948,17489,1,0,26947,17489,44436,3
315867,R160,02-03-00,66 ST-LINCOLN,2021-04-25,28817,15367,13,3,28804,15364,44168,6
261635,PTH03,00-01-08,JOURNAL SQUARE,2021-04-16,26237,17112,1,0,26236,17112,43348,4
...,...,...,...,...,...,...,...,...,...,...,...,...
316159,R160,02-05-01,66 ST-LINCOLN,2021-05-11,0,927,0,927,0,0,0,1
316158,R160,02-05-01,66 ST-LINCOLN,2021-05-10,0,927,0,927,0,0,0,0
316157,R160,02-05-01,66 ST-LINCOLN,2021-05-09,0,927,0,927,0,0,0,6
316156,R160,02-05-01,66 ST-LINCOLN,2021-05-08,0,927,0,927,0,0,0,5
